# Gradient Boosting Model inital attempt

In [40]:
!pip install xgboost

In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, classification_report

In [42]:
df= pd.read_csv("test.csv", sep= ",", dtype= object)
df

,constructorId,constructorRef,constructor_name,nationality_x,constructor_url,constructorStandingsId,raceId_x,constructor_pos,constructor_pos_txt,wins_x,...,code,dob,nationality_y,url,driverStandingsId,raceId_y,points,position,positionText,wins_y
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,26936.0,989.0,4.0,4,0.0,...,ALO,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,4,18,5.0,4,4,0
1,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,26936.0,989.0,4.0,4,0.0,...,ALO,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,12,19,6.0,7,7,0
2,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,26936.0,989.0,4.0,4,0.0,...,ALO,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,30,20,6.0,9,9,0
3,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,26936.0,989.0,4.0,4,0.0,...,ALO,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,51,21,6.0,10,10,0
4,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,26936.0,989.0,4.0,4,0.0,...,ALO,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,72,22,9.0,8,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355645,214,alpine,Alpine F1 Team,French,http://en.wikipedia.org/wiki/Alpine_F1_Team,28708.0,1120.0,6.0,6,0.0,...,GAS,1996-02-07,French,http://en.wikipedia.org/wiki/Pierre_Gasly,73126,1140,9.0,16,16,0
355646,214,alpine,Alpine F1 Team,French,http://en.wikipedia.org/wiki/Alpine_F1_Team,28708.0,1120.0,6.0,6,0.0,...,GAS,1996-02-07,French,http://en.wikipedia.org/wiki/Pierre_Gasly,73172,1141,26.0,12,12,0
355647,214,alpine,Alpine F1 Team,French,http://en.wikipedia.org/wiki/Alpine_F1_Team,28708.0,1120.0,6.0,6,0.0,...,GAS,1996-02-07,French,http://en.wikipedia.org/wiki/Pierre_Gasly,73218,1142,26.0,12,12,0
355648,214,alpine,Alpine F1 Team,French,http://en.wikipedia.org/wiki/Alpine_F1_Team,28708.0,1120.0,6.0,6,0.0,...,GAS,1996-02-07,French,http://en.wikipedia.org/wiki/Pierre_Gasly,73241,1143,36.0,11,11,0


In [43]:
df2= df.head(1000)
df2

,constructorId,constructorRef,constructor_name,nationality_x,constructor_url,constructorStandingsId,raceId_x,constructor_pos,constructor_pos_txt,wins_x,...,code,dob,nationality_y,url,driverStandingsId,raceId_y,points,position,positionText,wins_y
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,26936.0,989.0,4.0,4,0.0,...,ALO,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,4,18,5.0,4,4,0
1,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,26936.0,989.0,4.0,4,0.0,...,ALO,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,12,19,6.0,7,7,0
2,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,26936.0,989.0,4.0,4,0.0,...,ALO,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,30,20,6.0,9,9,0
3,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,26936.0,989.0,4.0,4,0.0,...,ALO,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,51,21,6.0,10,10,0
4,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,26936.0,989.0,4.0,4,0.0,...,ALO,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,72,22,9.0,8,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,26966.0,991.0,4.0,4,0.0,...,ALO,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,65052,865,76.0,1,1,1
996,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,26966.0,991.0,4.0,4,0.0,...,ALO,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,65076,866,86.0,2,2,1
997,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,26966.0,991.0,4.0,4,0.0,...,ALO,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,65100,867,111.0,1,1,2
998,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,26966.0,991.0,4.0,4,0.0,...,ALO,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,65124,868,129.0,1,1,2


In [44]:
df2.to_csv("help.csv", index=False)

In [45]:
#create binary variable for classifier
df['win']= (df['position']== 1).astype(int)

#features to keep for driver win predictions
features= [
    'grid', 'laps', 'results_points', 'fastestLapSpeed',  # race and performance info
    'AirTemp', 'Humidity', 'Pressure', 'TrackTemp', 'WindDirection', 'WindSpeed',  # weather/track conditions
    'Race_Entries', 'Race_Starts', 'Pole_Positions', 'Race_Wins', 'Podiums',
    'Fastest_Laps', 'Points', 'Pole_Rate', 'Start_Rate', 'Win_Rate', 'Podium_Rate',
    'FastLap_Rate', 'Points_Per_Entry', 'Years_Active'
]

#convert to numerical
for col in features:
    df[col] = pd.to_numeric(df[col], errors='coerce')

#only use rows will full data
df= df.dropna(subset=features+ ['win'])

X= df[features]
y= df['win']

In [46]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state= 33)

In [47]:
model= xgb.XGBClassifier(eval_metric='logloss', random_state=22, base_score= 0.5);

In [48]:
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [49]:
xgbYPred= model.predict(X_test)

In [50]:
print(accuracy_score(y_test, xgbYPred))
print(classification_report(y_test, xgbYPred))
# print(recall_score(y_test, xgbYPred))

race_winners= df.loc[X_test.index].copy()
race_winners['predicted_winner']= xgbYPred

winners= race_winners[race_winners['predicted_winner']== 1][['forename', 'surname']]
print("\n", winners)

1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     65819

    accuracy                           1.00     65819
   macro avg       1.00      1.00      1.00     65819
weighted avg       1.00      1.00      1.00     65819


 Empty DataFrame
Columns: [forename, surname]
Index: []
